In [1]:
DATA_DIR  = '/home/l-ashwin/Datasets/cifar-10-batches-py/'
UTILS_DIR = '/home/l-ashwin/Datasets/src_data/'

import sys
sys.path.append(UTILS_DIR)
from utils import load_cifar10

In [2]:
import numpy as np

In [3]:
import torch
import torch.nn as nn
from torchvision import transforms, models

# dataset

In [4]:
X_train, y_train, X_test, y_test = load_cifar10(DATA_DIR)

In [5]:
print(X_train.shape, y_train.shape)
print(X_test.shape,  y_test.shape )

(50000, 32, 32, 3) (50000,)
(10000, 32, 32, 3) (10000,)


In [6]:
class dataset:
    def __init__(self, data, target, transform=None):
        self.data = data
        self.target = target
        self.transform = transform
    
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        
        if self.transform:
            x = self.transform(x)
        
        return x, y
    
    def __len__(self):
        return len(self.data)

In [7]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0, 1)
])

In [8]:
train_data = dataset(X_train, y_train, transform)
test_data  = dataset(X_test , y_test , transform)

In [9]:
train_data[0][0].dtype

torch.float32

In [10]:
xx, yy = next(iter(train_data))
print(xx.shape, yy.shape)

torch.Size([3, 32, 32]) ()


In [11]:
BATCH_SIZE = 32

In [12]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader  = torch.utils.data.DataLoader(test_data,  batch_size=BATCH_SIZE)

In [13]:
xx, yy = next(iter(train_loader))
print(xx.shape, yy.shape)

torch.Size([32, 3, 32, 32]) torch.Size([32])


# model

In [14]:
model = models.resnet50(pretrained=True)
model.requires_grad_ = False

In [15]:
model.fc = nn.Sequential(
    nn.Linear(in_features=2048, out_features=64),
    nn.ReLU(),
    nn.Linear(in_features=64, out_features=10)
)
model.fc.requires_grad_ = True

# train

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [17]:
from torch.optim import Adam


criteria = nn.CrossEntropyLoss()
optim    = Adam(model.fc.parameters(), lr=1e-3)

In [24]:
def fit(model, optim, criteria, train_loader, EPOCHS):
    for epoch in range(EPOCHS):
        
        num_correct, loss_accume = 0, 0
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            
            model.train()
            optim.zero_grad()
            
            output  = model(imgs)
            _, pred = torch.max(output, dim=1)
            loss    = criteria(output, labels)
            
            loss.backward()
            optim.step()
            
            num_correct += sum(labels == pred).cpu().item()
            loss_accume += loss.cpu().item()
        
        print('EPOCH:{epoch} \t Loss:{loss} \t Accuracy:{accuracy}'.format(
            epoch    = epoch,
            loss     = np.round(loss_accume/len(train_loader.dataset), 2),
            accuracy = np.round(num_correct/len(train_loader.dataset), 2)
        ))

In [25]:
fit(model.to(device), optim, criteria, train_loader, 5)

EPOCH:0 	 Loss:0.04 	 Accuracy:0.49
EPOCH:1 	 Loss:0.04 	 Accuracy:0.5
EPOCH:2 	 Loss:0.04 	 Accuracy:0.5
EPOCH:3 	 Loss:0.04 	 Accuracy:0.51
EPOCH:4 	 Loss:0.04 	 Accuracy:0.51
